# **Proyecto Final**

<a href="https://colab.research.google.com/github/Mario-Carmona/ProyectoFinal-VC/blob/main/ProyectoFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
